In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# the pylj guide

Welcome to the guide for how to use pylj. The document will focus on how to get pylj running, the pedagogical aspects of the code, and give some examples of how pylj could be used in teaching of classical simulation to undergraduate chemistry and physics students. There is some assumed knowledge:

- you have some familiarity with the Jupyter notebooks framework (in that you were able to install Jupyter and open this notebook
- you have enough familiarity with Python to be able to install packages either via pip or from source. 

## how to get pylj?

The first thing to do is to get a copy of pylj running on your computer. pylj is available on the pypi package manager so it is possible to install pylj (and all of it's dependencies) using the following command,

```
pip install pylj 
```

in either the terminal (macOS or Linux) or the appropriate interface on Windows (e.g. Anaconda Prompt). It is also possible to install pylj from source by cloning the git repository and installing as follows (in the pylj main directory, 

```
pip install -r requirements.txt

python setup.py build

python setup.py install

python setup.py test
```

The former method is preferred as the latter will install the (possibly) unstable development version of pylj. 

## what is pylj?

Often is it difficult to introduce students (at both undergraduate and postgraduate level) to atomistic and molecular simulation due to the relatively high entry barrier (it is necessary to have some knowledge of lost arts such as the command line or programming). pylj aims to lower this barrier by allowing students to interact with a (simple) atomistic simulation with relative ease, while introducing ideas of programming. 

pylj is a simple tool that is capable of performing the two-dimensional simulation of particles interacting via a [Lennard-Jones potential](https://en.wikipedia.org/wiki/Lennard-Jones_potential), which essentially models the Paluli exclusion principle at short distances and the van der Waals attraction over long distances. The function used within pylj to determine the total energy of the system is as follows,

$$ E_{\text{total}}  = \begin{cases}
\sum_{\mathbf{r}}\bigg(\frac{A}{\mathbf{r}^{12}} - \frac{B}{\mathbf{r}^6}\bigg),& \text{if }\mathbf{r} < \text{cut-off}\\
0, & \text{otherwise}
\end{cases}
$$

where, $A = 1.363\times10^{-134}\;\text{Jm}^{12}$, $B = 9.273\times10^{-78}\;\text{Jm}^6$, and $\mathbf{r}$ is the distances between the pairs of particles (in pylj the cut-off is defaults to 15 Å, however may be varied). The values of $A$ and $B$ were taken from those determined for particles of argon by Rahman[1]. The result is a energy profile for a pair of particle that looks like this, 

In [ ]:
def e(r): 
    return 1.363e-134 / np.power(r, 12.) - 9.273e-78 / np.power(r, 6.)

r = np.linspace(3.1e-10, 8e-10, 100)
plt.plot(r, e(r))
plt.xlabel('$\mathbf{r}$/Å')
plt.ylabel('$E$/ J')
plt.show()

pylj is capable of running both molecular dynamics and Monte-Carlo simulations using built in functions, but could be extended to cover any simulation method that uses the Lennard-Jones potential. 

### visualisation

One of the most powerful aspects of pylj as a teaching tool is the visualisation environment. pylj is written to allow for the easy access to a series of commonly desired simulation output, such as instanteous pressure, temperature, and radial distribution funciton. These are all available through the use of a sampling class, these can either be those packaged with pylj, such as the Interactions or RDF classes, or can be custom built to the specification of the educator. More information about the sampling class and the sample module is available [here](http://pylj.readthedocs.io/en/latest/visualisation.html). 

## how to teach with pylj?

The current vision of pylj involves its application in a teaching laboratory setting (however there are may other applications available), and through this vision we have lead to two main areas that pylj could be used:

1. teaching about simulation algorithms, the mechanics of simulation and allowing the students to build a simulation themselves,
2. showing how simulation can be used to study physical attributes of chemical systems, consolidating learning introduced as lecture material. 

Some examples of these can be found in the [examples directory](./examples/), in particular the [molecular dynamics](./examples/molecular_dynamics/intro_to_molecular_dynamics.ipynb) and [Monte Carlo](./examples/monte_carlo/intro_to_monte_carlo.ipynb) examples show how pylj could be used in the teaching of molecular simulations, while the [ideal gas law](./examples/ideal_gas_law/ideal_gas_law.ipynb) notebook is examplary of how pylj can be used to present, in an interactive fashion, the deviation from the ideal gas law that is observed following the gas-to-liquid transition. 

1. [A. Rahman "Correlations in the Motion of Atoms in Liquid Argon", Physical Review 136 pp. A405–A411 (1964)](http://dx.doi.org/10.1103/PhysRev.136.A405)